
#### step0. 데이터 준비 
- 구글드라이브 마운트
- 데이터 : 

  1) 전국 소상공인 상권 정보 

  2) 네이버 음식점 리뷰 크롤링





In [ ]:
# 필요 라이브러리 임포트 
from google.colab import drive
import os, sys
import csv
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook

# 구글 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 데이터 로드 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tfb/shops_seoul.csv', encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (44,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# 필요한 데이터만 가져오기 
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

#### step1. 우리동네 음식점 찾기

In [ ]:
df = df.loc[(df['행정동명'] == '명동')] 
# df = df.loc[(df['행정동명'] == '명동') | (df['행정동명'] == '회현동')]

In [ ]:
# 컬럼명 수정 
df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

#### Step 2. 데이터 전처리 
 - 결측값 처리 
 - 음식점 분류 카테고리를 활용한 키워드리스트 생성
 - 코사인 유사도 구하기 

In [ ]:
# 결측치 처리 및 키워드리스트 생성 
df['cate_3'] = df['cate_3'].fillna('') 
# df['cate_mix'] = df['cate_1'] + df['cate_2'] +  df['cate_3'] 
df['cate_mix'] = df['name'] +' '+ df['cate_1'] +' '+ df['cate_2'] +' '+ df['cate_3'] # 키워드를 풍부하게 하기 위해 '상호명'도 추가함 
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")

In [ ]:
# 결측값 재확인 
df[df['cate_mix'].isnull()]
df['cate_mix'].isna().sum()
# df

0

In [ ]:
# 코사인 유사도 구하기 (키워드리스트 기반)
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2)) # TFIDF를 사용해도 될 듯.
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [ ]:
len(place_simi_cate) # 669
# place_simi_cate, place_simi_cate_sorted_ind

669

In [ ]:
place_simi_cate_sorted_ind

array([[  0, 297, 329, ..., 233, 516, 334],
       [  1, 396,  46, ..., 233, 516, 334],
       [  2, 297, 329, ..., 233, 516, 334],
       ...,
       [666, 274, 176, ..., 409, 408,   0],
       [667, 541, 222, ..., 411, 413,   0],
       [668,  57,  93, ..., 573, 269, 251]])

#### Step 3.  네이버 포털 블로그 리뷰, 별점 등의 데이터 크롤링 
 - 

In [ ]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install tqdm

In [ ]:
# -*- coding: UTF-8 -*-
 
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
 
#해당 url로 이동
url = "https://www.naver.com/" 
driver.get(url)
 
 # URL 테스트 
update = driver.find_element_by_css_selector('#NM_TS_ROLLING_WRAP > div > div')
print(update.text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


이슈
코로나바이러스감염증-19 현황


In [ ]:

# 크롤링 데이터를 담을 컬럼 생성
df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기
df['naver_map_url'] = ''


In [ ]:
# df.head(10)

In [ ]:
# 루프를 돌며 네이버 맵 URL 생성 

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # print("트라이", i)
        print(driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid'))
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                # print("예외1", i)
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                # print("예외2", i)
                time.sleep(1)
        else:
            pass


driver.quit()

이번에 찾을 키워드 : 0 / 668 행 명동%20맛생맛사


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if sys.path[0] == '':


33705879
이번에 찾을 키워드 : 1 / 668 행 명동%20한우방
1788142722
이번에 찾을 키워드 : 2 / 668 행 명동%20돌배기집


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
#18 0x55b0292c2b1a <unknown>
#19 0x55b0292dba41 <unknown>
#20 0x7fb1e8c9a6db <unknown>

이번에 찾을 키워드 : 7 / 668 행 명동%20라루체올리비아뷔페
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: headless chrome=101.0.4951.64)
Stacktrace:
#0 0x55b029283553 <unknown>
#1 0x55b028f6e183 <unknown>
#2 0x55b028fa4619 <unknown>
#3 0x55b028fa4831 <unknown>
#4 0x55b028fd9087 <unknown>
#5 0x55b028fc201d <unknown>
#6 0x55b028fd6d6c <unknown>
#7 0x55b028fc2363 <unknown>
#8 0x55b028f9847c <unknown>
#9 0x55b028f99945 <unknown>
#10 0x55b0292f39d0 <unknown>
#11 0x55b0292b8a38 <unknown>
#12 0x55b0292b875c <unknown>
#13 0x55b0292b8fc2 <unknown>
#14 0x55b0292f171b <unknown>
#15 0x55b0292b9221 <unknown>
#16 0x55b02929c5b3 <unknown>
#17 0x55b0292c2988 <unknown>
#18 0x55b0292c2b1a <unknown>
#19 0x55b0292dba41 <unk

In [ ]:
# df['naver_map_url']
df.head(10)

In [ ]:

# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줌 

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거
df = df.loc[~df['naver_map_url'].isnull()]

In [ ]:
# 최종 네이버맵URL 
df.naver_map_url

18          https://m.place.naver.com/restaurant/33705879
277       https://m.place.naver.com/restaurant/1788142722
533         https://m.place.naver.com/restaurant/38010908
1232        https://m.place.naver.com/restaurant/36868623
1573        https://m.place.naver.com/restaurant/21037913
                               ...                       
99229     https://m.place.naver.com/restaurant/1973976023
99639     https://m.place.naver.com/restaurant/1592955263
99883     https://m.place.naver.com/restaurant/1716296726
100724    https://m.place.naver.com/restaurant/1529023045
100772    https://m.place.naver.com/restaurant/1174608937
Name: naver_map_url, Length: 513, dtype: object

In [ ]:
# URL매핑된 데이터 저장 
df.to_csv("/content/drive/MyDrive/Colab Notebooks/tfb/result/rec_naver_url_myungdong.csv", mode='w')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
# -*- coding: UTF-8 -*-
# 네이버맵URL 기준으로 별점, 방문자리뷰수, 블로그리뷰수, 블로그리뷰텍스트 가져오기 
star_point_list = []
visit_review_qty_list = []
blog_review_qty_list = []
blog_review_text_list = [] 

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  time.sleep(2)
  driver.get(str(url))
  print(url, type(url))
  try:
    star_point = soup.select('div._20Ivz > span > em')[0].text.strip()
    blog_review_qty = soup.select('div._20Ivz > span > a > em')[1].text.strip()
    blog_review_text = soup.select('span.WoYOw')
    visit_review_qty = soup.select('div._20Ivz > span > a > em')[0].text.strip()

    star_point_list.append(star_point)
    blog_review_qty_list.append(blog_review_qty)
    blog_review_text_list.append(blog_review_text)
    visit_review_qty_list.append(visit_review_qty)

  except Exception as e1:
    print(f"{i}행 문제가 발생")
    star_point_list.append('null')
    blog_review_qty_list.append('null')    
    blog_review_text_list.append('null')
    visit_review_qty_list.append('null')
  
driver.quit()

df['star_point'] = star_point_list
df['blog_review_qty'] = blog_review_qty_list
df['blog_review_text'] = blog_review_text_list
df['visit_review_qty'] = visit_review_qty_list


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/513 [00:00<?, ?it/s]

https://m.place.naver.com/restaurant/33705879 <class 'str'>
0행 문제가 발생
https://m.place.naver.com/restaurant/1788142722 <class 'str'>
https://m.place.naver.com/restaurant/38010908 <class 'str'>
https://m.place.naver.com/restaurant/36868623 <class 'str'>
https://m.place.naver.com/restaurant/21037913 <class 'str'>
https://m.place.naver.com/restaurant/20397696 <class 'str'>
https://m.place.naver.com/restaurant/11808181 <class 'str'>
https://m.place.naver.com/restaurant/1750372339 <class 'str'>
https://m.place.naver.com/restaurant/1413027174 <class 'str'>
https://m.place.naver.com/restaurant/275788726 <class 'str'>
https://m.place.naver.com/restaurant/20888940 <class 'str'>
https://m.place.naver.com/restaurant/36561542 <class 'str'>
https://m.place.naver.com/restaurant/1194974888 <class 'str'>
https://m.place.naver.com/restaurant/20669049 <class 'str'>
https://m.place.naver.com/restaurant/11592650 <class 'str'>
https://m.place.naver.com/restaurant/19484724 <class 'str'>
https://m.place.naver

In [ ]:
# 리뷰, 별점 등을 포함하여 데이터 저장 
df.to_csv("/content/drive/MyDrive/Colab Notebooks/tfb/result/rec_naver_review_myungdong.csv", mode='w')

In [ ]:
df[df.star_point == 'null']

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,star_point,blog_review_qty,blog_review_text
18,맛생맛사,한식,한식/백반/한정식,한식 음식점업,명동,37.566984,126.988490,맛생맛사 한식 한식 백반 한정식 한식 음식점업,명동%20맛생맛사,https://m.place.naver.com/restaurant/33705879,null,null,null
5727,은&정명동닭갈비,한식,한식/백반/한정식,한식 음식점업,명동,37.564650,126.984198,은&정명동닭갈비 한식 한식 백반 한정식 한식 음식점업,명동%20은&정명동닭갈비,https://m.place.naver.com/restaurant/38582020,null,null,null
5943,김가네,분식,라면김밥분식,분식 및 김밥 전문점,명동,37.563614,126.985321,김가네 분식 라면김밥분식 분식 및 김밥 전문점,명동%20김가네,https://m.place.naver.com/restaurant/1477179302,null,null,null
7058,행화촌,중식,중국음식/중국집,중식 음식점업,명동,37.562114,126.981697,행화촌 중식 중국음식 중국집 중식 음식점업,명동%20행화촌,https://m.place.naver.com/restaurant/11727614,null,null,null
7251,버거킹,패스트푸드,패스트푸드,그외 기타 음식점업,명동,37.567398,126.979032,버거킹 패스트푸드 패스트푸드 그외 기타 음식점업,명동%20버거킹,https://m.place.naver.com/restaurant/1982878054,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99229,푸라닭,닭/오리요리,후라이드/양념치킨,치킨 전문점,명동,37.564719,126.983846,푸라닭 닭 오리요리 후라이드 양념치킨 치킨 전문점,명동%20푸라닭,https://m.place.naver.com/restaurant/1973976023,null,null,null
99639,광화문프렌즈,한식,한식/백반/한정식,한식 음식점업,명동,37.569002,126.975932,광화문프렌즈 한식 한식 백반 한정식 한식 음식점업,명동%20광화문프렌즈,https://m.place.naver.com/restaurant/1592955263,null,null,null
99883,쓰리펍,유흥주점,호프/맥주,기타 주점업,명동,37.558430,126.984984,쓰리펍 유흥주점 호프 맥주 기타 주점업,명동%20쓰리펍,https://m.place.naver.com/restaurant/1716296726,null,null,null
100724,리사르커피명동,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,명동,37.561774,126.988198,리사르커피명동 커피점 카페 커피전문점 카페 다방 비알콜 음료점업,명동%20리사르커피명동,https://m.place.naver.com/restaurant/1529023045,null,null,null


In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제
# df = df.loc[~(df['naver_store_type'].str.contains('null'))] # 임시 삭제

# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줌
# df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
df[['star_point', 'blog_review_qty']] = df[['star_point', 'blog_review_qty']].apply(pd.to_numeric)

ValueError: ignored

In [ ]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['blog_review_text']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

AttributeError: ignored

#### Step 4. 재료기반 음식점 리스트 추출 (Ex. '고등어' => '어촌생선구이굴밥', '아버지와고등어')

In [ ]:
# 앱에서 재료를 검색어로 입력하면 대응하는 음식점을 추출 
input = '돈까스'
df[df['cate_mix'].str.contains(input)]
print(df[df['cate_mix'].str.contains(input)])
# df_sort = df.sort_values(by='naver_star_point' ,ascending=False) # 네이버 평점 기준으로 가장 높은 음식점 선택 
df_sort = df.sort_values(by='lon' ,ascending=True).head(1) # 임시  
df_sort
df_sort.iloc[0]['name'].replace('\'','')
input_store = df_sort.iloc[0]['name']
input_store

             name cate_1  cate_2    cate_3 dong        lon         lat  \
1716         림돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.556728  126.985029   
12378     시청광장돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.565846  126.978759   
22455       림왕돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.556728  126.985029   
29209       다미돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.560637  126.987037   
41273     원조남산돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.557295  126.985995   
59440      남산왕돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.564719  126.983846   
65137  101번지남산돈까스     양식  돈가스전문점  서양식 음식점업   명동  37.556941  126.985728   

                            cate_mix    naver_keyword  \
1716         림돈까스 양식 돈가스전문점 서양식 음식점업        명동%20림돈까스   
12378     시청광장돈까스 양식 돈가스전문점 서양식 음식점업     명동%20시청광장돈까스   
22455       림왕돈까스 양식 돈가스전문점 서양식 음식점업       명동%20림왕돈까스   
29209       다미돈까스 양식 돈가스전문점 서양식 음식점업       명동%20다미돈까스   
41273     원조남산돈까스 양식 돈가스전문점 서양식 음식점업     명동%20원조남산돈까스   
59440      남산왕돈까스 양식 돈가스전문점 서양식 음식점업      명동%20남산왕돈까스   
65137  1

'림왕돈까스'

In [ ]:
df.head(2)

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,star_point,blog_review_qty,blog_review_text
18,맛생맛사,한식,한식/백반/한정식,한식 음식점업,명동,37.566984,126.988490,맛생맛사 한식 한식 백반 한정식 한식 음식점업,명동%20맛생맛사,https://m.place.naver.com/restaurant/33705879,null,null,null
277,한우방,한식,갈비/삼겹살,한식 음식점업,명동,37.565377,126.983916,한우방 한식 갈비 삼겹살 한식 음식점업,명동%20한우방,https://m.place.naver.com/restaurant/1788142722,4.34,9,[]


#### Step 6. 분석한 데이터를 종합하여 최종 알고리즘 생성
.공식 1 : 카테고리가 얼마나 유사한지

.공식 2 : 블로그 리뷰가 얼마나 유사한지

.공식 3 : 블로그 리뷰가 얼마나 많이 올라왔는지

.공식 4 : 블로그 별점이 얼마나 높은지

.공식 5 : 블로그 별점 평가가 얼마나 많이 됐는지

위 공식 5개를 적절한 비율로 가중치를 주면서 확인

In [ ]:
# 결측치 한번 더 제거 
df['blog_review_qty'] = df['blog_review_qty'].fillna(0)
df['star_point'] = df['star_point'].fillna(0)
# type(df['blog_review_qty'].values)

numpy.ndarray

In [ ]:
# 공식 1~5의 가중치 공식 
# * 0.003 등의 가중치를 줘서 조절
place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                #  + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                #  + np.repeat([df['blog_review_qty'].values], len(df['blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                #  + np.repeat([df['star_point'].values], len(df['star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                #  + np.repeat([df['star_point'].values], len(df['star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                #  + np.repeat([df['naver_star_point_qty'].values], len(df['naver_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )

In [ ]:
# df['star_point']

In [ ]:
# place_simi_co 

In [ ]:
df.head(6)

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,star_point,blog_review_qty,blog_review_text
18,맛생맛사,한식,한식/백반/한정식,한식 음식점업,명동,37.566984,126.988490,맛생맛사 한식 한식 백반 한정식 한식 음식점업,명동%20맛생맛사,https://m.place.naver.com/restaurant/33705879,null,null,null
277,한우방,한식,갈비/삼겹살,한식 음식점업,명동,37.565377,126.983916,한우방 한식 갈비 삼겹살 한식 음식점업,명동%20한우방,https://m.place.naver.com/restaurant/1788142722,4.34,9,[]
533,고려보쌈,한식,족발/보쌈전문,한식 음식점업,명동,37.567143,126.978708,고려보쌈 한식 족발 보쌈전문 한식 음식점업,명동%20고려보쌈,https://m.place.naver.com/restaurant/38010908,4.54,8,[[명동에서 14년째 최고급 한우를 합리적인 가격에 판매하는 한우전문점입니다.]]
1232,명동정한정식,한식,한식/백반/한정식,한식 음식점업,명동,37.561560,126.988450,명동정한정식 한식 한식 백반 한정식 한식 음식점업,명동%20명동정한정식,https://m.place.naver.com/restaurant/36868623,4.4,98,[]
1573,쯔루하시후게츠,일식/수산물,음식점-일식,일식 음식점업,명동,37.562828,126.985015,쯔루하시후게츠 일식 수산물 음식점-일식 일식 음식점업,명동%20쯔루하시후게츠,https://m.place.naver.com/restaurant/21037913,4.44,221,"[[명동역 10번출구 50m,전통인테리어 14개 개별실,대형 연회실,상견례, 돌잔치..."
1716,림돈까스,양식,돈가스전문점,서양식 음식점업,명동,37.556728,126.985029,림돈까스 양식 돈가스전문점 서양식 음식점업,명동%20림돈까스,https://m.place.naver.com/restaurant/20397696,4.49,"1,345",[[오사카에서 시작해 일본 전국에 현재 100여 개의 매장이 운영되고 있는 오코노미...


In [ ]:
# place_simi_co_sorted_ind 는 그냥 바로 사용
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 

In [ ]:
# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    print(place_name)
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    print("place_index: ", place_index )
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    print("place_title: ", place_title)
    print("place_index: ", place_index)
    print("similar_indexes: ", similar_indexes)
    return df.iloc[similar_indexes]

In [ ]:
# df.loc[22455]

name                                                            림왕돈까스
cate_1                                                             양식
cate_2                                                         돈가스전문점
cate_3                                                       서양식 음식점업
dong                                                               명동
lon                                                         37.556728
lat                                                        126.985029
cate_mix                                     림왕돈까스 양식 돈가스전문점 서양식 음식점업
naver_keyword                                              명동%20림왕돈까스
naver_map_url           https://m.place.naver.com/restaurant/20397696
star_point                                                       4.37
blog_review_qty                                                    35
blog_review_text    [[맘스터치 서울시청점은 고객님께 최상의 품질과 서비스를 제공하기 위해 항상 노력하...
Name: 22455, dtype: object

In [ ]:

df_result = find_simi_place(df, place_simi_co_sorted_ind, input_store, 5) # <= 재료 입력으로 변경 필요
df_result

림왕돈까스
place_index:  [22455]


IndexError: ignored

In [ ]:
## 추천 결과 저장
df_result.to_csv("/content/drive/MyDrive/Colab Notebooks/tfb/result/rec_result_myungdong.csv", mode='w')

#### Step 7. 지도상에 추천 음식점 시각화 

In [ ]:
!pip install haversine
from haversine import haversine, Unit
from folium.plugins import FastMarkerCluster
import pandas as pd
import folium

In [ ]:
MDLocation = (126.9858296,  37.55998)
a = 0
my_map = folium.Map(location=MDLocation, zoom_start=17)
for i in df_result.index :
  if(haversine(MDLocation, (df_result["lon"][i], df_result["lat"][i]), unit = 'km') < 0.5) :
    folium.Marker([df_result["lon"][i],  df_result["lat"][i]], tooltip=df_result["name"][i]).add_to(my_map)
    a = a + 1
    if a == 15 :
      break
folium.Marker(MDLocation, tooltip="내 위치").add_to(my_map)
my_map.save("./gdrive/MyDrive/TeamPotato/지도데이터/mymap.html")

my_map.get_root().render()
my_map

#### Step 8. 추천 시스템의 평가


1.   **Long tail plot** : 추천 이전의 인기도 측정
2.   **MAP@K(Mean Average Precision)** : "몇 개의 제품들을 고객에게 추천해줄 것인지"에서의 '몇 개'
3. **Coverage** : 추천 시스템이 Train 데이터로 학습한 결과, Train 데이터에 기반해 도출된 Ranked list에 있는 추천 제품들이 Test 데이터에서도 추천 제품인 퍼센티지를 의미
4. **Personalization** : 추천 시스템이 동일한 제품을 다른 유저들에게 똑같이 추천하는지 아니면 다르게 추천하는지 평가하기 위한 지표
5. **Intra-list Similarity** : 모든 추천 제품들 간의 평균 코사인 유사도


##### - 본 시스템은 서비스 시작 전이므로, 5번 평가 방법으로 평가를 진행함 



In [ ]:
# 모든 추천 제품들간 유사도 측정 
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))  
place_category = count_vect_category.fit_transform(df_result['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

print(len(place_simi_cate))
print(place_simi_cate_sorted_ind)

NameError: ignored

#### Next. 서비스 고도화 계획 


> 사용자 개개인에 대한 데이터를 확보하여 '개인화된 건강맛집 추천 서비스' 구현 예정

> 사용자 데이터는 서비스에 가입한 user 데이터(리뷰, 평점, 조회수 등)를 활용함 

